In [1]:
import pandas as pd
anime_full = pd.read_csv("Anime.csv")

In [2]:
anime_full = anime_full.replace("Unknown", pd.NA)
anime_full.Episodes = anime_full.Episodes.map(lambda x: int(x) if pd.notnull(x) else x)
anime_full.Score = anime_full.Score.map(lambda x: float(x) if pd.notnull(x) else x)
anime_full = anime_full.drop(anime_full.iloc[:, -10:], axis = 1)
anime_full = anime_full.drop(columns=["Japanese name", "Aired", "Producers", "Licensors", "Duration", "Ranked", "Popularity"])
anime_full

,MAL_ID,Name,Score,Genres,English name,Type,Episodes,Premiered,Studios,Source,Rating,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,TV,26,Spring 1998,Sunrise,Original,R - 17+ (violence & profanity),1251960,61971,105808,718161,71513,26678,329800
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,Movie,1,<NA>,Bones,Original,R - 17+ (violence & profanity),273145,1174,4143,208333,1935,770,57964
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,TV,26,Spring 1998,Madhouse,Manga,PG-13 - Teens 13 or older,558913,12944,29113,343492,25465,13925,146918
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,TV,26,Summer 2002,Sunrise,Original,PG-13 - Teens 13 or older,94683,587,4300,46165,5121,5378,33719
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,TV,52,Fall 2004,Toei Animation,Manga,PG - Children,13224,18,642,7314,766,1108,3394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17557,48481,Daomu Biji Zhi Qinling Shen Shu,<NA>,"Adventure, Mystery, Supernatural",<NA>,ONA,<NA>,<NA>,<NA>,Novel,<NA>,354,6,0,0,0,0,354
17558,48483,Mieruko-chan,<NA>,"Comedy, Horror, Supernatural",<NA>,TV,<NA>,<NA>,Passione,Manga,<NA>,7010,70,14,0,14,0,6982
17559,48488,Higurashi no Naku Koro ni Sotsu,<NA>,"Mystery, Dementia, Horror, Psychological, Supe...",Higurashi:When They Cry – SOTSU,TV,<NA>,Summer 2021,<NA>,Visual novel,R - 17+ (violence & profanity),11309,48,3,0,0,0,11306
17560,48491,Yama no Susume: Next Summit,<NA>,"Adventure, Slice of Life, Comedy",<NA>,TV,<NA>,<NA>,8bit,Manga,PG-13 - Teens 13 or older,1386,2,13,2,2,1,1368


In [3]:
anime_full.columns

Index(['MAL_ID', 'Name', 'Score', 'Genres', 'English name', 'Type', 'Episodes',
       'Premiered', 'Studios', 'Source', 'Rating', 'Members', 'Favorites',
       'Watching', 'Completed', 'On-Hold', 'Dropped', 'Plan to Watch'],
      dtype='object')

### 1. How many anime of each Source are in the Top 100 by Score?

In [4]:
source_in_top100 = anime_full.sort_values(by="Score", ascending=False).iloc[:100].groupby("Source").Score.count().sort_values(ascending=False).reset_index().rename(columns={"Score":"In Top 100"})

dif_list = list(set(anime_full.dropna(subset=["Source"]).Source.unique()).difference(set(source_in_top100.Source.unique())))
source_in_top100 = pd.concat([source_in_top100, pd.DataFrame(list(zip(dif_list, [0 for i in range(len(dif_list))])), columns=["Source", "In Top 100"], index=[j+7 for j in range(len(dif_list))])])

source_in_top100

,Source,In Top 100
0,Manga,64
1,Light novel,12
2,Original,10
3,Visual novel,5
4,Novel,4
5,Web manga,4
6,Other,1
7,Digital manga,0
8,Picture book,0
9,4-koma manga,0


### 2. How much is the number of people who Dropped the anime TV series related to the number of Episodes and the Score?

In [5]:
an2 = anime_full.dropna(subset=["Score", "Episodes", "Dropped", "Completed"]).astype({"Episodes": "int64", "Score": "float64"})
anime_tv = an2.loc[an2.Type == "TV"].loc[:,["Name", "Score", "Episodes", "Dropped", "Completed"]]
anime_tv["Sc-Ep Parameter"] = 10 - anime_tv["Score"] + anime_tv["Episodes"] / 182
anime_tv["Dropped %"] = anime_tv["Dropped"] * 100/ (anime_tv["Dropped"] + anime_tv["Completed"])
anime_tv["Sc-Ep-par levels"] = anime_tv["Sc-Ep Parameter"].map(lambda lvl: "Low" if lvl <= 2.72 else "Med-Low" if lvl <= 3.26 else "Med-High" if lvl <= 3.78 else "High")
anime_tv["Dropped % levels"] = anime_tv["Dropped %"].map(lambda lvl: "Low" if lvl <= 6.44 else "Med-Low" if lvl <= 11.38 else "Med-High" if lvl <= 19.6 else "High")
relation_lvl = anime_tv.apply(lambda row: row["Sc-Ep-par levels"] == row["Dropped % levels"], axis="columns").sum() / anime_tv.Name.count() * 100
print(f"{round(relation_lvl,2)}%")

49.91%


### 3. What are 10 Most productive studious?

In [6]:
studios_prod = anime_full.groupby(['Studios']).agg({'MAL_ID':['count'], 'Score': ['mean']})
studios_prod.columns = ['Quantity', 'Mean Score']
studios_prod["Mean diff"] = studios_prod["Mean Score"] - studios_prod["Mean Score"].mean()
studios_prod.sort_values(by="Quantity", ascending=False).iloc[:10]

,Quantity,Mean Score,Mean diff
Studios,,,
Toei Animation,755,6.725945,0.247026
Sunrise,479,6.905724,0.426804
J.C.Staff,358,6.826006,0.347086
Madhouse,324,6.969556,0.490636
TMS Entertainment,277,7.074408,0.595488
Production I.G,266,7.072996,0.594076
Studio Deen,263,6.975226,0.496306
Studio Pierrot,249,6.850046,0.371126
OLM,212,6.631658,0.152738


### 4. What percentage of anime has a particular Rating (G - All Ages, PG-13 - Teens 13 or older, R - 17+ (violence & profanity), ...)?

In [7]:
anime_full.groupby("Rating").apply(lambda df: df.shape[0] / anime_full.dropna(subset=["Rating"]).shape[0] * 100).reset_index().rename(columns={0:"%"}).sort_values(by="%", ascending=False)

,Rating,%
2,PG-13 - Teens 13 or older,36.339931
0,G - All Ages,34.265734
1,PG - Children,8.658291
5,Rx - Hentai,7.970843
3,R - 17+ (violence & profanity),6.856703
4,R+ - Mild Nudity,5.908498


### 5. Anime of what Main Genres are most likely to be completed?

In [8]:
anime_full["Main Genre"] = anime_full.dropna(subset=["Genres"]).apply(lambda row: row["Genres"].split(",")[0], axis="columns")
anime_full.groupby("Main Genre").apply(lambda df: round((df["Completed"] / (df["Dropped"] + df["Completed"]) * 100).mean(), 2)).reset_index().rename(columns={0:"Completed %"}).sort_values(by="Completed %", ascending=False)

,Main Genre,Completed %
39,Vampire,97.88
40,Yaoi,94.50
30,Shoujo,93.81
32,Shounen Ai,93.37
7,Ecchi,92.59
10,Harem,91.63
14,Josei,90.29
36,Super Power,89.59
38,Thriller,89.01
20,Music,88.94


### 6. Which studios have the biggest number of popular anime (With members > 100 000)?

In [9]:
an6 = anime_full.loc[anime_full.Members > 100000].groupby("Studios").Name.count().reset_index().rename(columns={"Name":"Popular anime number"}).sort_values(by="Popular anime number", ascending=False).merge(anime_full.groupby("Studios").Name.count().reset_index().rename(columns={"Name":"Anime number"}), on=["Studios"]).iloc[:10]
an6["%"] = round(an6["Popular anime number"] / an6["Anime number"] * 100, 2)
an6

,Studios,Popular anime number,Anime number,%
0,J.C.Staff,75,358,20.95
1,Madhouse,72,324,22.22
2,A-1 Pictures,72,193,37.31
3,Bones,52,127,40.94
4,Production I.G,45,266,16.92
5,Studio Deen,44,263,16.73
6,Kyoto Animation,42,115,36.52
7,Studio Pierrot,41,249,16.47
8,Toei Animation,36,755,4.77
9,Sunrise,36,479,7.52


### 7. Which anime have better average Score: new ( >= 2012) or old( < 2012)?

In [10]:
an7 = anime_full.dropna(subset=["Premiered", "Score", "Favorites"]).astype({"Score": "float64"})
an7["Old or New"] = an7.Premiered.map(lambda val: "Old" if int(val.split()[1]) < 2012 else "New")
old_avg = round(an7.loc[an7["Old or New"] == "Old"].Score.mean(),2)
new_avg = round(an7.loc[an7["Old or New"] == "New"].Score.mean(),2)
an7["Avg"] = an7["Old or New"].map(lambda val: old_avg if val == "Old" else new_avg)
new = an7.loc[an7["Old or New"] == "New"].groupby(["Old or New", "Avg", "Name"]).agg({"Score": ["sum"], "Favorites":["sum"]})
new.columns = ["Score", "Favorites"]
old = an7.loc[an7["Old or New"] == "Old"].groupby(["Old or New", "Avg", "Name"]).agg({"Score": ["sum"], "Favorites":["sum"]})
old.columns = ["Score", "Favorites"]
pd.concat([old.sort_values(by="Score", ascending = False), new.sort_values(by="Score")])

Score  Favorites
Old or New Avg  Name                                                  
Old        6.93 Fullmetal Alchemist: Brotherhood       9.19     183914
                Steins;Gate                            9.11     148452
                Hunter x Hunter (2011)                 9.10     147274
                Gintama'                               9.08       6567
                Clannad: After Story                   8.96      62162
...                                                     ...        ...
New        6.86 3-gatsu no Lion 2nd Season             9.00      11133
                Gintama': Enchousen                    9.04       2586
                Gintama°                               9.10      11868
                Shingeki no Kyojin Season 3 Part 2     9.10      40985
                Shingeki no Kyojin: The Final Season   9.17      44862

[3813 rows x 2 columns]

### 8. What types of anime are usually placed on-hold?

In [11]:
anime_full.groupby("Type").apply(lambda df: round((df["On-Hold"] / (df["On-Hold"] + df["Watching"]) * 100).mean(), 2)).reset_index().rename(columns={0:"On-Hold %"}).sort_values(by="On-Hold %", ascending=False)

,Type,On-Hold %
5,TV,38.49
4,Special,36.92
3,OVA,35.27
0,Movie,34.86
2,ONA,26.53
1,Music,23.99


### 9. Top Naruto anime by Favorites!

In [12]:
anime_full.dropna(subset=["English name"]).loc[anime_full.dropna(subset=["English name"])["English name"].map(lambda val: True if "Naruto" in val else False)].sort_values(by="Favorites", ascending=False)

,MAL_ID,Name,Score,Genres,English name,Type,Episodes,Premiered,Studios,Source,Rating,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Main Genre
1574,1735,Naruto: Shippuuden,8.16,"Action, Adventure, Comedy, Super Power, Martia...",Naruto:Shippuden,TV,500,Winter 2007,Studio Pierrot,Manga,PG-13 - Teens 13 or older,1543765,84651,362124,865643,109707,124253,82038,Action
10,20,Naruto,7.91,"Action, Adventure, Comedy, Super Power, Martia...",Naruto,TV,220,Fall 2002,Studio Pierrot,Manga,PG-13 - Teens 13 or older,1830540,65586,137167,1462223,61734,99806,69610,Action
12492,34566,Boruto: Naruto Next Generations,5.81,"Action, Adventure, Super Power, Martial Arts, ...",Boruto:Naruto Next Generations,TV,<NA>,Spring 2017,Studio Pierrot,Manga,PG-13 - Teens 13 or older,529732,5248,282596,26,67664,113677,65769,Action
7549,16870,The Last: Naruto the Movie,7.76,"Action, Super Power, Romance, Martial Arts, Sh...",The Last:Naruto the Movie,Movie,1,<NA>,Studio Pierrot,Manga,PG-13 - Teens 13 or older,352160,1592,4539,319967,860,677,26117,Action
9862,28755,Boruto: Naruto the Movie,7.5,"Action, Comedy, Martial Arts, Shounen, Super P...",Boruto:Naruto the Movie,Movie,1,<NA>,Studio Pierrot,Manga,PG-13 - Teens 13 or older,320603,1029,5595,285754,999,1215,27040,Action
7021,13667,Naruto: Shippuuden Movie 6 - Road to Ninja,7.67,"Action, Adventure, Super Power, Martial Arts, ...",Road to Ninja:Naruto the Movie,Movie,1,<NA>,Studio Pierrot,Manga,PG-13 - Teens 13 or older,223826,442,2621,202852,793,589,16971,Action
2267,2472,Naruto: Shippuuden Movie 1,7.29,"Action, Adventure, Comedy, Fantasy, Shounen",Naruto:Shippuden the Movie,Movie,1,<NA>,Studio Pierrot,Manga,PG-13 - Teens 13 or older,211544,394,2172,194744,656,523,13449,Action
3516,4134,"Naruto: Shippuuden - Shippuu! ""Konoha Gakuen"" Den",7.15,Comedy,Naruto Shippuden:Konoha Gakuen - Special,Special,1,<NA>,Studio Pierrot,<NA>,PG-13 - Teens 13 or older,52243,331,847,41631,445,420,8900,Comedy
414,442,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,7.1,"Adventure, Comedy, Historical, Supernatural, D...",Naruto the Movie:Ninja Clash in the Land of Snow,Movie,1,<NA>,Studio Pierrot,Manga,PG-13 - Teens 13 or older,215046,303,1551,202775,589,476,9655,Adventure
3639,4437,Naruto: Shippuuden Movie 2 - Kizuna,7.29,"Action, Martial Arts, Shounen, Supernatural",Naruto:Shippuden the Movie 2 -Bonds-,Movie,1,<NA>,Studio Pierrot,Manga,PG-13 - Teens 13 or older,188680,278,1728,171637,680,500,14135,Action


### 10. Anime recommendations!

In [13]:
an10 = anime_full.dropna(subset=["Score", "Members", "Favorites", "Watching", "Completed", "Plan to Watch"]).astype({"Score": "float64"})
an10["Completed %"] = an10["Completed"] * 100/ (an10["Dropped"] + an10["Completed"]) 
for col in ["Score", "Members", "Favorites", "Watching", "Completed %", "Plan to Watch"]:
    high_quat = an10[col].describe(percentiles=[.25,.5,.9]).loc["90%"]
    an10[f"{col} levels"] = an10[col].map(lambda val: "High" if val >= high_quat else "Low")

def all_high(row):
    if all([v == "High" for v in row.values]):
        return True
    return False
an10.loc[an10.iloc[:, -6:].apply(all_high, axis=1)].sort_values(by="Score", ascending=False)

,MAL_ID,Name,Score,Genres,English name,Type,Episodes,Premiered,Studios,Source,...,Dropped,Plan to Watch,Main Genre,Completed %,Score levels,Members levels,Favorites levels,Watching levels,Completed % levels,Plan to Watch levels
14963,38524,Shingeki no Kyojin Season 3 Part 2,9.10,"Action, Drama, Fantasy, Military, Mystery, Sho...",Attack on Titan Season 3 Part 2,TV,10,Spring 2019,Wit Studio,Manga,...,3099,103563,Action,99.659422,High,High,High,High,High,High
9886,28851,Koe no Katachi,9.00,"Drama, School, Shounen",A Silent Voice,Movie,1,<NA>,Kyoto Animation,Manga,...,1949,199329,Drama,99.831050,High,High,High,High,High,High
11308,32281,Kimi no Na wa.,8.96,"Romance, Supernatural, School, Drama",Your Name.,Movie,1,<NA>,CoMix Wave Films,Original,...,2463,216535,Romance,99.831832,High,High,High,High,High,High
12933,35247,Owarimonogatari 2nd Season,8.93,"Mystery, Comedy, Supernatural, Vampire",Owarimonogatari Second Season,TV,7,Summer 2017,Shaft,Light novel,...,1697,94913,Mystery,98.944639,High,High,High,High,High,High
2656,2904,Code Geass: Hangyaku no Lelouch R2,8.91,"Action, Military, Sci-Fi, Super Power, Drama, ...",Code Geass:Lelouch of the Rebellion R2,TV,25,Spring 2008,Sunrise,Original,...,12463,139305,Action,98.838699,High,High,High,High,High,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7879,18397,Shingeki no Kyojin OVA,7.83,"Action, Super Power, Drama, Fantasy, Shounen",<NA>,OVA,3,<NA>,Wit Studio,Manga,...,2210,39409,Action,99.085458,High,High,High,High,High,High
1535,1689,Byousoku 5 Centimeter,7.73,"Drama, Romance, Slice of Life",5 Centimeters Per Second,Movie,3,<NA>,CoMix Wave Films,Original,...,4580,147860,Drama,99.133765,High,High,High,High,High,High
13935,36896,Boku no Hero Academia the Movie 1: Futari no Hero,7.70,"Action, Shounen, Super Power",My Hero Academia:Two Heroes,Movie,1,<NA>,Bones,Manga,...,870,76245,Action,99.746434,High,High,High,High,High,High
11110,31765,Sword Art Online Movie: Ordinal Scale,7.65,"Action, Game, Adventure, Romance, Fantasy",Sword Art Online the Movie -Ordinal Scale-,Movie,1,<NA>,A-1 Pictures,Original,...,1966,83716,Action,99.524974,High,High,High,High,High,High
